In [6]:
list('nl')

['n', 'l']

In [2]:
import sys

In [3]:
import getopt

In [5]:
print(getopt.__doc__)

Parser for command line options.

This module helps scripts to parse the command line arguments in
sys.argv.  It supports the same conventions as the Unix getopt()
function (including the special meanings of arguments of the form `-'
and `--').  Long options similar to those supported by GNU software
may be used as well via an optional third argument.  This module
provides two functions and an exception:

getopt() -- Parse command line options
gnu_getopt() -- Like getopt(), but allow option and non-option arguments
to be intermixed.
GetoptError -- exception (class) raised with 'opt' attribute, which is the
option involved with the exception.



In [115]:
getopt.gnu_getopt?

### Process text

In [6]:
filename='testtext.txt'

In [7]:
with open(filename) as f:
    textstring=f.read()

In [8]:
import nltk.data

In [9]:
sent_detector=nltk.data.load('tokenizers/punkt/english.pickle')

In [10]:
#print('\n-----\n'.join(sent_detector.tokenize(textstring.strip())))

Because of smartphone apps like Uber, taxi politics have gotten a lot of press lately.
-----
One of the claims Uber proponents make is that normal taxis do not serve everybody.
-----
In New York City, taxis are heavily regulated.
-----
There are only about 14,000 taxis allowed to pick up passengers off the street in the center of the city.
-----
Uber gets around this regulation by only picking up passengers who request a ride on the app.
-----
Because there are few taxis, critics say that they can get away with only serving the few.
-----
Some claim that taxis only serve those with the privilege to live in the city center, and that they are biased against people of color.
-----
As a New Yorker who has spent my fair share of time waiting in taxi traffic on my bicycle, I wanted to get some understanding about this issue.
-----
Are taxis actually unfair, as critics claim?


In [11]:
sent_list=sent_detector.tokenize(textstring.strip(),realign_boundaries=False)

In [12]:
import string

In [13]:
def remove_stuff(st,remove):
    import string
    return st.translate(string.maketrans('',''),remove)

In [14]:
def remove_all(st):
    return remove_stuff(st,string.digits+string.punctuation)

In [15]:
def process_string(st):
    return remove_all(st.replace('-',' ')).lower().split()

In [16]:
words_sent_list=map(process_string,sent_list)

### Get functional and frequency data

In [17]:
import numpy as np

In [18]:
from equiv_sylb import equiv_sylb
#equiv_sylb=e.equiv_sylb

In [19]:
equiv_sylb(50)

1.044970334824336

In [20]:
import pickle

In [21]:
with open('frequency_dict.pkl','rb') as f:
    freq_ser=pickle.load(f)

### Compute 

In [22]:
def get_frequency(word,ser):
    if ser.has_key(word):
        return ser[word]
    else:
        return 1


In [23]:
just_words=[item for sublist in words_sent_list for item in sublist]

In [24]:
frequency=np.array(map(lambda x: get_frequency(x,freq_ser),just_words))

In [25]:
total_words=len(just_words)
total_sent=len(words_sent_list)
total_sylb=equiv_sylb(frequency).sum()

In [26]:
from __future__ import division

In [27]:
def flesch_kincaid_gl(words,sent,sylb):
    return 0.39*(words/sent)+11.8*(sylb/words)-15.59

In [28]:
improved_flesch_gl=flesch_kincaid_gl(total_words,total_sent,total_sylb)
improved_flesch_gl

8.9534532897635515

### Compute problem sentences

In [29]:
n_sent=2

In [31]:
def get_sent_freq(sent,freq_ser):
    return map(lambda x: get_frequency(x,freq_ser),sent)

In [40]:
sent_freq=map(lambda x: np.array(get_sent_freq(x,freq_ser)),words_sent_list)

In [44]:
words_per_sent=map(lambda x: x.shape[0], sent_freq)

In [49]:
def get_sent_sylb(freq_list):
    return sum(map(lambda x: equiv_sylb(x),freq_list))

In [53]:
sylb_per_sent=map(lambda x: get_sent_sylb(x),sent_freq)

In [54]:
imp_flesch_sent=map(lambda x,y: flesch_kincaid_gl(x,1,y),words_per_sent,sylb_per_sent)

In [58]:
top_arg=np.argpartition(np.array(imp_flesch_sent),-n_sent)[-n_sent:]

In [61]:
top_sent=map(lambda x: sent_list[x], top_arg)

In [63]:
top_flesch=map(lambda x: imp_flesch_sent[x], top_arg)

In [98]:
sent_print=map(lambda x: '%1.0f\t\t: %s' % x, zip(top_flesch,top_sent))

In [109]:
full_document_string='document grade level: %1.0f' % improved_flesch_gl

In [107]:
sentence_string=('grade level \t: sentence\n'+('-'*40)+'\n'+(('\n'+('-'*40)+'\n').join(sent_print)))

In [112]:
print(full_document_string+'\n'+'-'*80+'\n\n'+sentence_string)

document grade level: 9
--------------------------------------------------------------------------------

grade level 	: sentence
----------------------------------------
17		: Because of smartphone apps like Uber, taxi politics have gotten a lot of press lately.
----------------------------------------
20		: Are taxis actually unfair, as critics claim?
